<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Algorithme de Needleman et Wunsch

## Version Itérative

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Les coûts

Reprenons les fonctions de coûts comme on les avait définies dans la séquence précédente&nbsp;:

In [ ]:
# la fonction d'insertion la plus simple possible
def insertion_cost(base):
    return 1

In [ ]:
# la fonction de substitution la plus simple possible
def substitution_cost(base1, base2):
    return 1 if base1 != base2 else 0

*******

In [ ]:
def needleman_wunsch_iter(adn1, adn2):
    max_i = len(adn1)
    max_j = len(adn2)

    print("max", max_i, max_j)
    ### phase 1
    for sum in range(max_i + max_j + 1):
        print("sum", sum)
        for i in range(sum+1):
#            print("i", i)
            j = sum - i
            if 0 <= i <= max_i and 0 <= j <= max_j:
                print(i,j)

In [ ]:
needleman_wunsch_iter("abcd", "abcdef")